<a href="https://colab.research.google.com/github/god12070521-prog/fantastic-octo-rotary-phone/blob/main/HTML_to_PDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!apt-get update
!apt-get install -y \
    libatk1.0-0 \
    libatk-bridge2.0-0 \
    libcups2 \
    libxkbcommon0 \
    libxcomposite1 \
    libxdamage1 \
    libxfixes3 \
    libxrandr2 \
    libgbm1 \
    libpango-1.0-0 \
    libcairo2 \
    libasound2 \
    libnspr4 \
    libnss3 \
    libxss1 \
    fonts-liberation \
    libappindicator3-1 \
    xdg-utils \
    libdrm2 \
    libx11-xcb1 \
    libxcb-dri3-0 \
    libxshmfence1

!pip install playwright
!playwright install chromium
!playwright install-deps chromium


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://cli.github.com/packages stable/main amd64 Packages [345 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,202 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,532 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu j

In [14]:
from playwright.async_api import async_playwright
from pathlib import Path
from google.colab import files
import asyncio
import nest_asyncio

nest_asyncio.apply()


async def html_to_pdf_async(source: str, output: str = "output.pdf", wait_time: int = 0):
    async with async_playwright() as p:
        browser = await p.chromium.launch(
            headless=True,
            args=['--no-sandbox', '--disable-setuid-sandbox']
        )
        page = await browser.new_page()

        source_path = Path(source)
        if source_path.exists():
            await page.goto(f"file://{source_path.absolute()}")
        else:
            await page.goto(source)

        await page.wait_for_load_state("networkidle")

        if wait_time > 0:
            await page.wait_for_timeout(wait_time)

        # 페이지 전체 크기 가져오기
        dimensions = await page.evaluate('''() => {
            return {
                width: Math.max(
                    document.body.scrollWidth,
                    document.documentElement.scrollWidth,
                    document.body.offsetWidth,
                    document.documentElement.offsetWidth
                ),
                height: Math.max(
                    document.body.scrollHeight,
                    document.documentElement.scrollHeight,
                    document.body.offsetHeight,
                    document.documentElement.offsetHeight
                )
            }
        }''')

        # 콘텐츠 크기 그대로 한 페이지로 출력 (margin 없이)
        await page.pdf(
            path=output,
            width=f"{dimensions['width']}px",
            height=f"{dimensions['height']}px",
            print_background=True,
            margin={
                "top": "0",
                "bottom": "0",
                "left": "0",
                "right": "0"
            }
        )

        await browser.close()
        print(f"PDF 생성 완료: {output}")
        print(f"크기: {dimensions['width']}px x {dimensions['height']}px")

    return output


def html_to_pdf(source: str, output: str = "output.pdf", wait_time: int = 0):
    return asyncio.get_event_loop().run_until_complete(
        html_to_pdf_async(source, output, wait_time)
    )


In [15]:
uploaded = files.upload()
filename = list(uploaded.keys())[0]

html_to_pdf(filename, "converted.pdf")
files.download("converted.pdf")


Saving page5.html to page5.html
PDF 생성 완료: converted.pdf
크기: 1280px x 720px


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>